## Shortcut Creation in Fabric Lakehouse

In [ ]:
#Shortcut Raw Files/ from one Lakehouse to the same Lakehouse Tables/

#A shortcut connection was already made in the lakehouse - it will use that by default (this is where you config which blob storage to connect to)

import requests
from azure.identity import ClientSecretCredential, DefaultAzureCredential

# ✅ Service Principal credentials
tenant_id = "<Tenant Id here>"
client_id = "<Client Id here>"
client_secret = "<Client Secret here>"

# ✅ Authenticate and get token
scope = "https://api.fabric.microsoft.com/.default"
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token(scope).token

# ✅ Target Lakehouse details
workspace_id = "<Workspace Id here>"  # Target workspace GUID #UnifiedData
lakehouse_id = "<Lakehouse Id here>"  # Target Lakehouse GUID #maag_silver

# ✅ Source Lakehouse details (replace with real GUIDs)
source_workspace_id = "<Source Workspace Id here>" #UnifiedData
source_lakehouse_id = "<Source Lakehouse Id here>" #maag_bronze

# ✅ API endpoint with conflict policy
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts?shortcutConflictPolicy=CreateOrOverwrite"

# ✅ Shortcut payload
payload = {
    "path": "Files/Bronze_Raw_Files/sales",  # Folder in target Lakehouse where shortcut will appear
    "name": "ModelCardShortcut",
    "target": {
        "oneLake": {
            "workspaceId": source_workspace_id,
            "itemId": source_lakehouse_id,
            "path": "Files/Raw_Files/sales" #need to be a folder in Files/ - do not a folder that is shortcutted already
        }
    }
}



headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# ✅ Call API
response = requests.post(url, headers=headers, json=payload)

# ✅ Check result
if response.status_code in [200, 201]:
    print("✅ Shortcut created successfully!")
    print(response.json())
else:
    print(f"❌ Failed to create shortcut: {response.status_code}")
    print(response.text)

## Make bulk transformation shortcuts using API - didn't seem to work - public preview

https://learn.microsoft.com/en-us/rest/api/fabric/core/onelake-shortcuts/creates-shortcuts-in-bulk?tabs=HTTP#create-bulk-shortcut-example

In [ ]:

import requests
from azure.identity import ClientSecretCredential, DefaultAzureCredential

# ✅ Service Principal credentials
tenant_id = "<Tenant Id here>"
client_id = "<Client Id here>"
client_secret = "<Client Secret here>"

# ✅ Authenticate and get token
scope = "https://api.fabric.microsoft.com/.default"
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token(scope).token

# ✅ Target Lakehouse details
workspace_id = "<Workspace Id here>"  # Target workspace GUID #UnifiedData
lakehouse_id = "<Lakehouse Id here>"  # Target Lakehouse GUID #maag_silver

# ✅ Source Lakehouse details (replace with real GUIDs)
source_workspace_id = workspace_id 
source_lakehouse_id = lakehouse_id 

# ✅ API endpoint with conflict policy
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts/bulkCreate?shortcutConflictPolicy=CreateOrOverwrite"

# ✅ Shortcut payload with CSV-to-Delta transform
payload = {
  "createShortcutRequests": [
    {  
      "path": "Files/raw_csv",                        # Shortcut location in TARGET lakehouse
      "name": "table1",
      "target": {
        "oneLake": {
          "workspaceId": source_workspace_id,      # ID of the SOURCE workspace
          "itemId": source_lakehouse_id,           # ID of the SOURCE lakehouse
          "path": "Tables/dbo/table1"#,                     # SOURCE folder containing CSV files
          #"connectionId": "719c7460-72e1-4e6a-bacc-731192baee5b"   # Omit if not needed
        }
      },
      "transform": {
        "type": "csvToDelta",
        "properties": {
          "delimiter": ",",
          "skipFilesWithErrors": True,
          "useFirstRowAsHeader": True
               }
      }
    }
  ]
}


headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# ✅ Call API
response = requests.post(url, headers=headers, json=payload)
print(response)



<Response [202]>


## "get" a shortcut transformation

In [ ]:
import requests
from azure.identity import ClientSecretCredential

# Service Principal credentials
tenant_id = "<Tenant Id here>"
client_id = "<Client Id here>"
client_secret = "<Client Secret here>"

# Authenticate and get token
scope = "https://api.fabric.microsoft.com/.default"
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token(scope).token

# Target Lakehouse details
workspace_id = "<Workspace Id here>"  # Target workspace GUID #UnifiedData
lakehouse_id = "<Lakehouse Id here>"  # Target Lakehouse GUID #maag_silver


# ✅ API endpoint for GET (retrieve shortcut details)
shortcut_path = "Tables/dbo/raw_csv"
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{lakehouse_id}/shortcuts/{shortcut_path}"

headers = {
    "Authorization": f"Bearer {token}"
}

# ✅ Perform GET request
response = requests.get(url, headers=headers)

# ✅ Check result
if response.status_code == 200:
    print("✅ Shortcut details retrieved successfully!")
    print(response.json())
else:
    print(f"❌ Failed to retrieve shortcut: {response.status_code}")
    print(response.text)